In [1]:
from bs4 import BeautifulSoup
import requests
import time
from random import randint
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}

In [172]:
def scroll_bottom():
    SCROLL_PAUSE_TIME = 1

    while True:

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:

            # try again (can be removed)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # check if the page height has remained the same
            if new_height == last_height:
                # if so, you are done
                break
            # if not, move on to the next loop
            else:
                last_height = new_height
                SCROLL_PAUSE_TIME += 0.01
                continue

In [173]:
def get_wine_url(base_url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS660DQ1WSwYSLmoFQNn0NNuyxKLM1JLEHLX8JNuixJLMvPTi-OT80rwStXzblNTiZLXykuhYoFIwZQShjKE8EwhtDqFM1IptkxMB-20m0g%3D%3D&cart_item_source=nav-explore'):
    #initialize driver
    chromedriver = "/Applications/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    driver.get(base_url)
    
    scroll_bottom()
    
    #get links
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    wine_page_links = [link.get('href').strip() for link in soup.find_all('a', class_='anchor__anchor--2QZvA')][::4]
    return wine_page_links

In [180]:
wine_page_links = get_wine_url()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=60970): Max retries exceeded with url: /session/b800ae1e1a99afbda8199788e34dff29/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1352a7bb0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [178]:
wine_page_links

['/veuve-clicquot-brut-carte-jaune-champagne/w/1128385?year=N.V.&cart_item_source=direct-explore',
 '/moet-chandon-imperial-brut-champagne/w/1122095?year=N.V.&cart_item_source=direct-explore',
 '/casal-garcia-vinho-verde-branco/w/1226459?year=N.V.&cart_item_source=direct-explore',
 '/farnese-edizione-cinque-autoctoni/w/21646?year=N.V.&cart_item_source=direct-explore',
 '/piccini-memoro-rosso-tuscany/w/1155726?year=N.V.&cart_item_source=direct-explore',
 '/louis-roederer-brut-premier-champagne/w/74298?year=N.V.&cart_item_source=direct-explore',
 '/bollinger-special-cuvee-brut-ay-champagne/w/18929?year=N.V.&cart_item_source=direct-explore',
 '/taittinger-brut-reserve-champagne/w/1129619?year=N.V.&cart_item_source=direct-explore',
 '/mateus-the-original-rose/w/1253734?year=N.V.&cart_item_source=direct-explore',
 '/perrier-jouet-grand-brut-champagne/w/79160?year=N.V.&cart_item_source=direct-explore',
 '/laurent-perrier-brut-cuvee-champagne-rose/w/8305?year=N.V.&cart_item_source=direct-expl

In [99]:
def parse_ratings(soup):
    rating_raw = soup.find_all("div", class_=lambda value: value and value.startswith("vivinoRatingWide"))[0]
    rating_clean = rating_raw.text.split()[0]
    return rating_clean


In [138]:
def parse_winery(soup):
    winery_raw = soup.find(class_="winery")
    winery_clean = winery_raw.text.strip('\n')
    return winery_clean

In [141]:
def parse_wine_name(soup):
    name_raw = soup.find(class_="vintage")
    name_raw2 = name_raw.text.split()[:-1]
    name_clean = ' '.join(name_raw2)
    return name_clean

In [102]:
def parse_wine_vintage(soup):
    vintage_raw = soup.find(class_="vintage")
    vintage_clean = vintage_raw.text.split()[-1]
    return vintage_clean

In [103]:
def parse_wine_type(soup):
    type_raw = soup.find(class_="wineLocationHeader__wineType--14nrC")
    type_clean = type_raw.text
    return type_clean

In [104]:
def parse_wine_region(soup):
    region_raw = soup.find(class_="anchor__anchor--3DOSm wineLocationHeader__region--1cbip")
    region_clean = region_raw.text
    return region_clean

In [105]:
def parse_wine_country(soup):
    country_raw = soup.find(class_="anchor__anchor--3DOSm wineLocationHeader__country--1RcW2")
    country_clean = country_raw.text
    return country_clean

In [106]:
def parse_number_reviews(soup):   
    reviews_raw = soup.find(class_="vivinoRatingWide__basedOn--s6y0t")
    reviews_clean = reviews_raw.text[:-8]
    return reviews_clean

In [134]:
def parse_avg_price(soup):
    price_raw = soup.find_all("span", class_=lambda value: value and value.startswith("purchaseAvailability"))[0]
    price_clean = price_raw.text[1:]
    return price_clean

In [142]:
def scrape_wine_info():
    avg_rating_list = []
    winery_list = []
    name_list = []
    vintage_list =[]
    type_list = []
    region_list = []
    country_list = []
    num_reviews_list = []
    avg_price_list = []

    base_url = 'https://www.vivino.com'
    chromedriver = "/Applications/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    
    for page in wine_page_links[:5]:
        driver.get(base_url + page)
        soup = BeautifulSoup(driver.page_source, 'html.parser')      

        try:
            rating = parse_ratings(soup)
            avg_rating_list.append(rating)
        except:
            avg_rating_list.append('error')
            
        try:
            winery = parse_winery(soup)
            winery_list.append(winery)
        except:
            winery_list.append('error')
        
        try:
            name = parse_wine_name(soup)
            name_list.append(name)
        except:
            name_list.append('error')
            
        try:
            year = parse_wine_vintage(soup)
            vintage_list.append(year)
        except:
            vintage_list.append('error')
            
        try:
            wine_type = parse_wine_type(soup)
            type_list.append(wine_type)
        except:
            type_list.append('error') 
            
        try:
            region = parse_wine_region(soup)
            region_list.append(region)
        except:
            region_list.append('error')   
        
        try:
            country = parse_wine_country(soup)
            country_list.append(country)
        except:
            country_list.append('error')
        
        try:
            num_reviews = parse_number_reviews(soup)
            num_reviews_list.append(num_reviews)
        except:
            num_reviews_list.append('error')
            
        try:
            avg_price = parse_avg_price(soup)
            avg_price_list.append(avg_price)
        except:
            avg_price_list.append('error')
         
            
    features = ('Average Rating', 'Winery', 'Wine Name', 'Vintage',
                'Type', 'Region', 'Country', 'Number of Reviews',
               'Average Price')
    df = pd.DataFrame(list(zip(avg_rating_list, winery_list, name_list, vintage_list, type_list, region_list, 
                               country_list, num_reviews_list, avg_price_list)), columns =features) 
    
    return df

In [150]:
scrape_wine_info()

,Average Rating,Winery,Wine Name,Vintage,Type,Region,Country,Number of Reviews,Average Price
0,5.041,Real Companhia Velha,Colheita Port,1944,Fortified Wine,Porto,Portugal,41,416.21
1,5.041,,Bi-Centenaire Reserve Societeit Phoenix Delft,1847,Red wine,Haut-Médoc,France,41,16.72
2,5.032,Château d'Yquem,Sauternes,1945,Dessert wine,Sauternes,France,32,error
3,4.945,Moët & Chandon,Les Vignes de Saran Champagne Grand Cru 'Choui...,1992,Sparkling wine,Champagne Grand Cru 'Chouilly',France,45,1241.54
4,4.9107,Dom Pérignon,P2 Plénitude Brut Champagne,1995,Sparkling wine,Champagne,France,107,error
